In [1]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import Point
from shapely.geometry import shape
import geopandas as gpd
from osgeo import gdal
import numpy as np
import os
import requests

# USDA Data Calories Work
After some exploration, we decided to use the [Zenodo data](https://zenodo.org/record/7332106#.ZCHhtuxuerM) instead.

In [2]:
# from Zenodo data
kcal_conversion = pd.read_csv("../unsynced-data/Stability_Crop_Diversity-2.0/Data/Inputs/kcal_conversion.csv")
kcal_conversion

,Crop,Crop_Class,Crop_Utilization,Crop_Production,Production_Units,Conversion_to_kg,kcal_kg
0,CAULIFLOWER,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,98.0
1,ASPARAGUS,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,106.0
2,LETTUCE,HEAD,FRESH MARKET,ALL PRODUCTION PRACTICES,CWT,45.35920,114.0
3,LETTUCE,HEAD,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,CWT,45.35920,114.0
4,LETTUCE,LEAF,FRESH MARKET,ALL PRODUCTION PRACTICES,CWT,45.35920,140.0
...,...,...,...,...,...,...,...
109,COTTON,UPLAND,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,480 LB BALES,217.68480,NaN
110,HAY,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,TONS,907.18500,NaN
111,HOPS,ALL CLASSES,ALL UTILIZATION PRACTICES,ALL PRODUCTION PRACTICES,LB,0.45351,NaN
112,SORGHUM,ALL CLASSES,SILAGE,ALL PRODUCTION PRACTICES,TONS,907.18500,NaN


Made this request and downloaded all crop production data by county:

https://quickstats.nass.usda.gov/api/api_GET/?key=A5DB238B-8B04-3675-8A54-F373FA342D6F&source_desc=CENSUS&sector_desc=CROPS&agg_level_desc=COUNTY&domain_desc=TOTAL&statisticcat_desc=PRODUCTION&year__GE=2017&format=CSV

In [3]:
# all crop data from USDA API by county in 2017
usda = pd.read_csv("../unsynced-data/usda_allcrops_county_2017.csv")
usda

,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,unit_desc,short_desc,state_ansi,state_fips_code,state_alpha,state_name,asd_code,asd_desc,county_ansi,county_code,county_name,location_desc,Value,CV (%)
0,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",1,1,AL,ALABAMA,60,WIREGRASS,69.0,69,HOUSTON,"ALABAMA, WIREGRASS, HOUSTON",1,0.4
1,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",4,4,AZ,ARIZONA,80,SOUTHERN,27.0,27,YUMA,"ARIZONA, SOUTHERN, YUMA",4,82.1
2,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",5,5,AR,ARKANSAS,60,EAST CENTRAL,85.0,85,LONOKE,"ARKANSAS, EAST CENTRAL, LONOKE",1,0.2
3,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",6,6,CA,CALIFORNIA,40,CENTRAL COAST,79.0,79,SAN LUIS OBISPO,"CALIFORNIA, CENTRAL COAST, SAN LUIS OBISPO",1,(L)
4,CROP TOTALS,CROP TOTALS,ALL CLASSES,PRODUCTION CONTRACT,ALL UTILIZATION PRACTICES,OPERATIONS,"CROP TOTALS, PRODUCTION CONTRACT - OPERATIONS ...",6,6,CA,CALIFORNIA,80,SOUTHERN CALIFORNIA,65.0,65,RIVERSIDE,"CALIFORNIA, SOUTHERN CALIFORNIA, RIVERSIDE",1,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46558,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,1.0,1,ADAMS,"WASHINGTON, EAST CENTRAL, ADAMS","735,000",8.9
46559,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,21.0,21,FRANKLIN,"WASHINGTON, EAST CENTRAL, FRANKLIN",(D),(D)
46560,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",53,53,WA,WASHINGTON,50,EAST CENTRAL,25.0,25,GRANT,"WASHINGTON, EAST CENTRAL, GRANT","318,000",8.9
46561,VEGETABLES,SWEET CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SEED,LB,"SWEET CORN, SEED - PRODUCTION, MEASURED IN LB",55,55,WI,WISCONSIN,50,CENTRAL,1.0,1,ADAMS,"WISCONSIN, CENTRAL, ADAMS",(D),(D)


In [4]:
# how many unique values in each column of usda
for c in usda.columns:
    print(c, len(usda[c].unique()))
    

group_desc 3
commodity_desc 48
class_desc 38
prodn_practice_desc 2
util_practice_desc 8
unit_desc 8
short_desc 87
state_ansi 50
state_fips_code 50
state_alpha 50
state_name 50
asd_code 23
asd_desc 86
county_ansi 267
county_code 269
county_name 1774
location_desc 3056
Value 24967
CV (%) 569


In [5]:
# create a new column in counties for each unique value in the commodity_desc column
for i in range(len(usda["commodity_desc"].unique())):
    counties[usda["commodity_desc"].unique()[i]] = 0


NameError: name 'counties' is not defined

In [ ]:
for i in range(len(usda["commodity_desc"].unique())):
    counties[usda["commodity_desc"].unique()[i]+"_unit"] = 0

In [ ]:
# for each of the rows in usda add the value of the value column to the column in counties that matches the commodity_desc in the row where usda county_code and counties COUNTYFP match
for i in range(len(usda)):
    if usda["county_code"][i] == counties["COUNTYFP"][i]:
        counties[usda["commodity_desc"][i]][i] = usda["value"][i]        